# Aviary 0.0.5

##### Config

In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
from __future__ import print_function, division
#path = "data/state-farm/"
path = "data/state-farm/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
batch_size=64

###### Set up batches

In [4]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [5]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filename) = get_classes(path)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


In [6]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                                 shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 1500 images belonging to 10 classes.


###### Load data from images (slow)

In [72]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


###### Load/Save data with array

In [73]:
save_array(path+'results/val.dat', val)
save_array(path+'results/trn.dat', trn)

In [7]:
val = load_array(path+'results/val.dat')
trn = load_array(path+'results/trn.dat')

### Model

#### Build

In [8]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dropout(0.1),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=8, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.01
    model.fit_generator(batches, batches.nb_sample, nb_epoch=12, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.1
    model.fit_generator(batches, batches.nb_sample, nb_epoch=16, validation_data=val_batches, 
                         nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 1
    model.fit_generator(batches, batches.nb_sample, nb_epoch=16, validation_data=val_batches, 
                         nb_val_samples=val_batches.nb_sample)
    return model

#### Run 0.0.6

In [75]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 34s - loss: 2.9844 - acc: 0.1253 - val_loss: 2.3778 - val_acc: 0.1040
Epoch 2/2
1500/1500 [==============================] - 28s - loss: 2.7432 - acc: 0.1560 - val_loss: 2.2816 - val_acc: 0.1530
Epoch 1/8
1500/1500 [==============================] - 34s - loss: 2.4548 - acc: 0.2040 - val_loss: 2.4117 - val_acc: 0.1620
Epoch 2/8
1500/1500 [==============================] - 26s - loss: 2.3739 - acc: 0.2233 - val_loss: 2.5295 - val_acc: 0.1730
Epoch 3/8
1500/1500 [==============================] - 32s - loss: 2.1661 - acc: 0.2760 - val_loss: 2.5975 - val_acc: 0.1750
Epoch 4/8
1500/1500 [==============================] - 28s - loss: 2.0995 - acc: 0.3100 - val_loss: 2.6254 - val_acc: 0.1760
Epoch 5/8
1500/1500 [==============================] - 28s - loss: 2.0135 - acc: 0.3333 - val_loss: 2.6011 - val_acc: 0.1890
Epoch 6/8
1500/1500 [==============================] - 27s - loss: 1.9100 - acc: 0.3707 - val_loss: 2.5602 - val_acc: 0.1940


#### 0.0.7

lowered the learning rate for the initial training

In [9]:
model = conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.5941 - acc: 0.1933 - val_loss: 2.9657 - val_acc: 0.1110
Epoch 2/2
1500/1500 [==============================] - 26s - loss: 1.8740 - acc: 0.3827 - val_loss: 2.1067 - val_acc: 0.2240
Epoch 1/8
1500/1500 [==============================] - 33s - loss: 1.5833 - acc: 0.4813 - val_loss: 2.0865 - val_acc: 0.2120
Epoch 2/8
1500/1500 [==============================] - 26s - loss: 1.4581 - acc: 0.5120 - val_loss: 2.1686 - val_acc: 0.2550
Epoch 3/8
1500/1500 [==============================] - 26s - loss: 1.3412 - acc: 0.5440 - val_loss: 2.2776 - val_acc: 0.2330
Epoch 4/8
1500/1500 [==============================] - 26s - loss: 1.2418 - acc: 0.5893 - val_loss: 2.3216 - val_acc: 0.2450
Epoch 5/8
1500/1500 [==============================] - 27s - loss: 1.1302 - acc: 0.6193 - val_loss: 2.2952 - val_acc: 0.2430
Epoch 6/8
1500/1500 [==============================] - 28s - loss: 1.1285 - acc: 0.6340 - val_loss: 2.2841 - val_acc: 0.1980


In [10]:
model.optimizer.lr = 1
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 33s - loss: 0.2321 - acc: 0.9340 - val_loss: 0.2595 - val_acc: 0.9170
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.2222 - acc: 0.9347 - val_loss: 0.2365 - val_acc: 0.9250
Epoch 3/4
1500/1500 [==============================] - 26s - loss: 0.2264 - acc: 0.9327 - val_loss: 0.2931 - val_acc: 0.8970
Epoch 4/4
1500/1500 [==============================] - 28s - loss: 0.2265 - acc: 0.9360 - val_loss: 0.2195 - val_acc: 0.9380


In [11]:
model.optimizer.lr = 10
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 35s - loss: 0.2105 - acc: 0.9487 - val_loss: 0.2287 - val_acc: 0.9240
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.2015 - acc: 0.9433 - val_loss: 0.2157 - val_acc: 0.9370
Epoch 3/4
1500/1500 [==============================] - 28s - loss: 0.2267 - acc: 0.9387 - val_loss: 0.2233 - val_acc: 0.9350
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 0.2125 - acc: 0.9433 - val_loss: 0.2322 - val_acc: 0.9220


In [12]:
model.optimizer.lr = 1000
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 32s - loss: 0.2173 - acc: 0.9460 - val_loss: 0.2099 - val_acc: 0.9370
Epoch 2/4
1500/1500 [==============================] - 27s - loss: 0.2016 - acc: 0.9447 - val_loss: 0.2672 - val_acc: 0.9120
Epoch 3/4
1500/1500 [==============================] - 27s - loss: 0.1887 - acc: 0.9527 - val_loss: 0.2558 - val_acc: 0.9210
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 0.1916 - acc: 0.9440 - val_loss: 0.2071 - val_acc: 0.9460


In [13]:
model.optimizer.lr = 1000000
model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)

Epoch 1/4
1500/1500 [==============================] - 33s - loss: 0.1904 - acc: 0.9500 - val_loss: 0.2885 - val_acc: 0.8910
Epoch 2/4
1500/1500 [==============================] - 28s - loss: 0.1992 - acc: 0.9433 - val_loss: 0.1970 - val_acc: 0.9430
Epoch 3/4
1500/1500 [==============================] - 27s - loss: 0.1920 - acc: 0.9487 - val_loss: 0.2359 - val_acc: 0.9320
Epoch 4/4
1500/1500 [==============================] - 27s - loss: 0.1755 - acc: 0.9600 - val_loss: 0.2326 - val_acc: 0.9240


### 0.0.8

In [ ]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dropout(0.1),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.01
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.1
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                         nb_val_samples=val_batches.nb_sample)
    return model
model = conv1(batches)

### Save weights

In [34]:
model.save_weights(path+'results/aviary_005.h5')

# Predict & Confusion Matrix

In [40]:
val_batches, probs = model.test(path+'valid', batch_size = batch_size)

AttributeError: 'Sequential' object has no attribute 'test'

In [39]:
probs = probs[:,0]
preds = np.round(1-probs)
probs[:8]

NameError: name 'probs' is not defined

In [37]:
labels = val_batches.classes
filenames = val_batches.filenames

In [38]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels, preds)
plot_confusion_matrix(cm, val_batches.class_indices)

NameError: name 'preds' is not defined

###### ___archive_imagenet